# 서울특별시 일반음식점 데이터 과제

## 설정

In [1]:
# Jupyter Notebook 설정 : 모든 결과 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
# DeprecationWarning / FutureWarning(차후 버전에서 없어질 기능이니 미리 경고하는 것) 끄기
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# matplotlib에서 한글 사용 설정
from matplotlib import font_manager, rc
import matplotlib as mpl
import platform

mpl.rcParams['axes.unicode_minus'] = False

system = platform.system()
if system == "Windows":
    font_path = "C:/Windows/Fonts/malgun.ttf"
elif system == "Darwin": # Mac OS
    font_path = "/System/Library/Fonts/AppleSDGothicNeo.ttc"
elif system == "Linux":
    font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
else:
    font_path = None

if font_path:
    font = font_manager.FontProperties(fname=font_path).get_name()
    mpl.rcParams['font.family'] = font
else:
    print("폰트 경로를 확인해주세요.")

In [4]:
# plt.show() 동작 방법 변경 설정 (window >> jupyter notebook)
%matplotlib inline

## 준비

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json

df = pd.read_csv("서울특별시_일반음식점.csv", encoding='cp949', usecols=['번호','상세영업상태코드','상세영업상태명','업태구분명','위생업태명','소재지전체주소','도로명전체주소'])
print(type(df.columns), len(df.columns))
print(type(df.index), len(df.index))
df.head()

<class 'pandas.core.indexes.base.Index'> 7
<class 'pandas.core.indexes.range.RangeIndex'> 472915


,번호,상세영업상태코드,상세영업상태명,소재지전체주소,도로명전체주소,업태구분명,위생업태명
0,1,1,영업,서울특별시 강서구 마곡동 757 두산더랜드파크 B동 207호,"서울특별시 강서구 마곡중앙로 161-8, 두산더랜드파크 B동 2층 207호 (마곡동)",한식,한식
1,2,1,영업,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,한식
2,3,1,영업,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,한식
3,4,1,영업,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,한식
4,5,1,영업,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,일식


## 데이터 확인

In [35]:
# NaN 데이터 확인
len(df[df['번호'].isnull()]) # 0개
len(df[df['상세영업상태코드'].isnull()]) # 0개
len(df[df['상세영업상태명'].isnull()]) # 0개
len(df[df['소재지전체주소'].isnull()]) # 237개
len(df[df['도로명전체주소'].isnull()]) # 24,9613개
len(df[df['업태구분명'].isnull()]) # 21개
len(df[df['위생업태명'].isnull()]) # 28개

# 전체 데이터 개수
df['번호'].count() # 472,915개

# '소재지전체주소'와 '도로명전체주소' 둘 다 NaN인 데이터 개수
len(df[(df['소재지전체주소'].isnull()) & (df['도로명전체주소'].isnull())])

0

0

0

237

249613

21

28

np.int64(472915)

0

### 영업 상태 확인을 위한 컬럼 확인 ('상세영업상태명':''상세영업상태코드')
- '영업' : 1
- '폐업' : 2

In [36]:
df.groupby('상세영업상태코드').count()
df.groupby('상세영업상태명').count()

,번호,상세영업상태명,소재지전체주소,도로명전체주소,업태구분명,위생업태명
상세영업상태코드,,,,,,
1,124208,124208,124124,122519,124197,124191
2,348707,348707,348554,100783,348697,348696


,번호,상세영업상태코드,소재지전체주소,도로명전체주소,업태구분명,위생업태명
상세영업상태명,,,,,,
영업,124208,124208,124124,122519,124197,124191
폐업,348707,348707,348554,100783,348697,348696


### 업종을 판단하기 위한 기준 컬럼 확인
- '업태구분명' 컬럼으로 업종 판단 ('업태구분명' 컬럼 보다 데이터가 많음)
- 치킨집으로 분류 가능한 업태구분명 : 통닭(치킨), 호프/통닭

In [37]:
# NaN 데이터 개수 확인
len(df[df['업태구분명'].isnull()]) # 21개
len(df[df['위생업태명'].isnull()]) # 28개

# 치킨집으로 분류 가능한 업태구분명 찾기
df.groupby('업태구분명').count() # 통닭(치킨), 호프/통닭

21

28

,번호,상세영업상태코드,상세영업상태명,소재지전체주소,도로명전체주소,위생업태명
업태구분명,,,,,,
193959.150482967,1,1,1,1,1,0
간이주점,1,1,1,1,0,1
감성주점,345,345,345,345,345,345
경양식,47881,47881,47881,47856,17715,47881
기타,32944,32944,32944,32911,28852,32944
기타 휴게음식점,3,3,3,3,2,3
김밥(도시락),1532,1532,1532,1532,1155,1532
까페,7906,7906,7906,7889,6072,7906
냉면집,120,120,120,120,118,120


## 데이터 전처리
1. 주소 데이터에서 구와 동 데이터 추출하여 새로운 컬럼 추가
    - '소재지전체주소' NaN 데이터 237개, '도로명전체주소' NaN 데이터 24,9613개 (둘 다 NaN인 경우는 없음)  
    - '소재지전체주소' 컬럼 기준으로 처리하고 NaN의 경우는 '도로명전체주소' 컬럼에서 데이터를 가져와서 처리
    - '소재지전체주소'는 ' '으로 구분시 시, 구, 동, ... 으로 구성
    - '도로명전체주소'는 ' '으로 구분되고 시, 구, ... 으로 구성, 동은 마지막에 '(동)'으로 구성

In [38]:
gu_list = [] # 구
dong_list = [] # 동
index_list = [] # index

# '도로명전체주소'로부터 '구'와 '동' 데이터 추출
def append_address_data(row_key):
    row_value = df['도로명전체주소'][row_key]
    if (bool(row_value) and (isinstance(row_value, str))):
        i = row_value.find('(')
        row_value_l = row_value[:i]
        row_value_r = row_value[i:]
        item_list = row_value_l.split(" ")
        dong = row_value_r
        icomma = dong.find(',')
        if (bool(dong) and (isinstance(dong, str)) and (len(dong) > 1) and (dong[0] =='(') and (dong[-1] ==')')):
            gu_list.append(item_list[1])
            dong = dong[1:icomma] if (icomma > 0) else dong[1:-1]
            dong_list.append(dong)
            index_list.append(row_key)
        else:
            print("[NG1]", row_key, row_value)

address_column_series = df['소재지전체주소']
for row_key, row_value in address_column_series.items():
    if (bool(row_value) and (isinstance(row_value, str))):
        item_list = row_value.split(" ")
        if (len(item_list) > 2):
            gu_list.append(item_list[1])
            dong_list.append(item_list[2])
            index_list.append(row_key)
        else:
            # 1건 발생 : 66952 143-15번지 지상1층
            print("[NG2]", row_key, row_value)
            append_address_data(row_key)
    else:
        append_address_data(row_key)

df['구'] = pd.Series(gu_list, index=index_list, name='구')
df['동'] = pd.Series(dong_list, index=index_list, name='동')
with open('preprocess_result.csv', 'w', encoding='cp949', newline='') as f:
    df.to_csv(f, index=False)

[NG2] 66952 143-15번지 지상1층


## 결과

### 서울시 도봉구에서 치킨집이 가장 많은 동은?
- 창동이 146개로 가장 많음 (폐업한 치킨집은 제외)

In [39]:
s = (df['구'] == '도봉구') & (df['상세영업상태코드'] == 1) & ((df['업태구분명'] == '통닭(치킨)') | (df['업태구분명'] == '호프/통닭'))
df[s].groupby('동').count().sort_values('번호', ascending=False)

,번호,상세영업상태코드,상세영업상태명,소재지전체주소,도로명전체주소,업태구분명,위생업태명,구
동,,,,,,,,
창동,146,146,146,146,146,146,146,146
방학동,123,123,123,123,123,123,123,123
쌍문동,85,85,85,85,85,85,85,85
도봉동,77,77,77,77,77,77,77,77


### 서울시 구별 치킨집 현황 (폐업한 치킨집은 제외)
- 구로구가 783개로 가장 많음
- 용산구가 255개로 가장 적음

In [40]:
s = (df['상세영업상태코드'] == 1) & ((df['업태구분명'] == '통닭(치킨)') | (df['업태구분명'] == '호프/통닭'))
df[s].groupby('구').count().sort_values('번호', ascending=False)

,번호,상세영업상태코드,상세영업상태명,소재지전체주소,도로명전체주소,업태구분명,위생업태명,동
구,,,,,,,,
구로구,783,783,783,783,770,783,783,783
마포구,745,745,745,743,734,745,745,745
영등포구,730,730,730,729,711,730,730,730
관악구,694,694,694,693,694,694,694,694
중랑구,663,663,663,663,663,663,663,663
광진구,650,650,650,650,644,650,650,650
강서구,625,625,625,625,623,625,625,625
강동구,605,605,605,605,603,605,605,605
노원구,586,586,586,586,584,586,586,586


# END